In [1]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

## Указать все пути
MODEL_NAME - путь к предобученной модели (уже указан)  
TEXT_FILES_DIR - путь к директории с текстами  
FAISS_INDEX_PATH - путь для сохранения/загрузки FAISS-индекса  
FILENAMES_PATH - путь для сохранения/загрузки списка имен файлов

In [2]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
TEXT_FILES_DIR = "garant"
FAISS_INDEX_PATH = "results_BERT_Default/text_index.faiss"
FILENAMES_PATH = "results_BERT_Default/filenames.npy"

In [3]:
model = SentenceTransformer(MODEL_NAME)
index = faiss.read_index(FAISS_INDEX_PATH)
filenames = np.load(FILENAMES_PATH, allow_pickle=True)

## Функция поиска похожих текстов

In [4]:
def find_similar_texts_from_file(test_file_path, top_k=5):
    with open(test_file_path, "r", encoding="utf-8") as f:
        test_text = f.read().strip()

    texts = []
    filenames_local = []
    for file in sorted(os.listdir(TEXT_FILES_DIR)):
        if file.endswith(".txt"):
            file_path = os.path.join(TEXT_FILES_DIR, file)
            with open(file_path, "r", encoding="utf-8") as f:
                texts.append(f.read().strip())
            filenames_local.append(file)
    
    query_embedding = model.encode([test_text], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)

    print("Наиболее похожие документы:")
    for i, idx in enumerate(indices[0], start=1):
        print(f"{i}. Файл: {filenames_local[idx]} (Дистанция: {distances[0][i-1]:.4f})")
        print(texts[idx][:300] + "...")
        print("-" * 80)

## Тестирование
file_path - путь к текстовому файлу, который будет поступать на вход

In [5]:
file_path = "garant.txt"

In [6]:
find_similar_texts_from_file(file_path, top_k=20)

Наиболее похожие документы:
1. Файл: garant_1261.txt (Дистанция: 0.0000)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Международного дня инвалидов в городе Хабаровске в 2004 году"

В связи с проведением 3 декабря 2004 года Международного дня инвалидов и в целях привлечения внимания широких слоев общественности города, предприятий, учрежден...
--------------------------------------------------------------------------------
2. Файл: garant_1425.txt (Дистанция: 4.3260)
﻿
Постановление Главы города Челябинска 
от 8 июня 2004 г. N 991-п 
"Об утверждении "Программы мероприятий, обеспечивающих доступ инвалидов и
маломобильных групп населения в существующие здания общественного 
назначения" 

В целях реализации постановления Главы города от 18.02.2003г. N 240-п "О бесп...
--------------------------------------------------------------------------------
3. Файл: garant_0078.txt (Дистанция: 4.5565)
﻿
Постановление Мэра г. Хабаровска
от 12 ноября 2004 г. N 1662
"О г